In [1]:
import requests
import os
from dotenv import load_dotenv
import pymongo
import time

In [2]:
def get_current_timestamp_milliseconds():
  """
  Returns the current timestamp in milliseconds since the epoch.
  """
  return int(time.time() * 1000)

In [3]:
load_dotenv() 
LI_URI = os.environ['LI_URI']
LI_VERSION = os.environ['LI_VERSION']
LI_ACCESS_TOKEN = os.environ['LI_ACCESS_TOKEN']

In [4]:
db_client = pymongo.MongoClient('mongodb://localhost:27017')
db = db_client['db_li_page_posts']
tb_page_post = db['tb_page_posts']

In [5]:
def post_2_page(payload):
    #print(url)
    url = LI_URI + 'posts'
    headers = {
        'Authorization': LI_ACCESS_TOKEN,
        'LinkedIn-Version': LI_VERSION,
        'Content-Type': 'application/json'
    }
    try:
        detail = requests.post(url, json=payload, headers=headers)
        return detail   #LI does not return response detail
    except Exception as e:
        print(e)
        return {'error': e}

In [6]:
#get 1 latest post and repost to LI
def get_1_latest_pots():
    latest_post = tb_page_post.find_one({'shared': 0}, sort=[('lastModifiedAt', -1)])
    return latest_post
#test
#end

In [14]:
#get 1 random post to reshare if there were less than 5 posts in last 24 hours
def get_1_latest_post():
    timenow = get_current_timestamp_milliseconds()  #milliseconds
    last24hours = timenow - 24 * 60 * 60 * 1000
    todayPosts = tb_page_post.count_documents({'shared': 1, '$and': [ {'shared_time': {'$gt': last24hours }}, {'shared_time': {'$lt': timenow }} ] })
    # print(last24hours)
    print(todayPosts)
    if todayPosts < 10:
        #2. if today posted < 5 posts:
        #2.1 get 1 new post RANDOMLY, sorted by lastModifiedAt
        random_document = next(tb_page_post.aggregate([
            {"$match": {'shared': 0}},
            {"$sort": {"lastModifiedAt": -1}},
            #{"$sort": [("lastModifiedAt", -1)]},
            {"$sample": {"size": 1}}
        ]))
        return random_document
    #
    return None

In [21]:
the_post = get_1_latest_post()
the_post

5


{'_id': ObjectId('67f8da55c729960221dc56fd'),
 'id': 'urn:li:share:7315681063110856706',
 'lastModifiedAt': 1744194297975,
 'author': '103249511',
 'description': 'Master Excel with ChatGPT!\n\n\nCredit to: @[Adam Biddlecombe](urn:li:person:9PeQcNGyIG)\n\n\nOriginal post below\n\n\n⬇️⬇️⬇️\n\n"Master Excel with ChatGPT!\n\nExcel and Google Sheets are powerful tools for data analysis and management, but mastering them can be challenging. \n\nEnter ChatGPT, your AI assistant to help you become a spreadsheet pro!\n\nHere’s how ChatGPT can help you with your Excel and Google Sheets skills:\n\nFormula Creation:\n- Generate complex formulas\n- Explain formula logic\n- Troubleshoot formula errors\n\nData Analysis:\n- Suggest analysis methods\n- Interpret data trends\n- Create pivot tables\n\nVisualisation Tips:\n- Choose the right chart type\n- Format charts for clarity\n- Create dynamic dashboards\n\nAutomation & Macros:\n- Write VBA code for Excel\n- Create Google Apps Scripts\n- Explain aut

In [22]:
#log the time when the post is shared
def update_shared_info(post_id, post_desc):
    tb_page_post.update_one({'id': post_id}, {'$set': {'shared': 1, 'shared_time': get_current_timestamp_milliseconds()}})
    print('Finished reshare to the page with description: ' + post_desc + ' ...')

In [23]:
#repost exactly, do not download (maximum 5 repost per day)
def auto_repost(random_document):
    #2 post it to page
    if random_document != None:
        payload = {
            "author": "urn:li:organization:" + os.environ['MY_PAGE_ID'],    #post to my page
            "commentary": '',   #the_post['description'], #duplicated content
            "visibility": "PUBLIC",
            "distribution": {
                "feedDistribution": "MAIN_FEED"
            },
            "lifecycleState": "PUBLISHED",
            "reshareContext": {
                "parent": random_document['id']
            }
        }
        #print(payload)
        result = post_2_page(payload)
        # print(result)
        if 'error' not in result:
            #2.3 Update to db: shared=1, shared_time=now
            update_shared_info(random_document['id'], random_document['description'][:30])

In [ ]:
#find the image link and reshare in LI page
def reshare_img(li_img_id):
    #find img details
    url = LI_URI + 'images/' + li_img_id
    print(url)
    headers = {
        'Authorization': LI_ACCESS_TOKEN,
        'LinkedIn-Version': LI_VERSION
    }
    try:
        detail = requests.get(url, headers=headers)
        #print(detail.json())
        #if 'downloadUrl' in detail:

        
    except Exception as e:
        print(e)

In [25]:
#download image and re-share into the page
def download_n_reshare_post():
    if the_post is None:
        return  #reach limit for last 24 hours, do not share anything
    #check type of the post
    if 'urn:li:image:' in the_post['media']:
        #download the image
        img_link = reshare_img(the_post['media'])

    else:
        #reshare the video, do not download
        auto_repost(the_post)

In [32]:
download_n_reshare_post()

https://api.linkedin.com/rest/images/urn:li:image:D5622AQF9_VQXCminsg
{'downloadUrl': 'https://media.licdn.com/dms/image/v2/D5622AQF9_VQXCminsg/feedshare-shrink_1280/B56ZYaFKUvHQAo-/0/1744194283763?e=1747872000&v=beta&t=i3jcvNE_T2u-vVACWBsvEAAeApeLharSfmjSYqMcmGw', 'owner': 'urn:li:organization:103249511', 'id': 'urn:li:image:D5622AQF9_VQXCminsg', 'downloadUrlExpiresAt': 1747872000000, 'status': 'AVAILABLE'}


In [8]:
#db_client.close()